In [1]:
import sys
sys.path.append("..") 

In [2]:
#!conda install -c conda-forge tensorboard

In [3]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from pathlib import Path
import os
import torch.cuda
import albumentations as albu
import segmentation_models_pytorch as smp
from  segmentation_models_pytorch.utils.base import Metric
from segmentation_models_pytorch.base.modules import Activation
from collections import defaultdict
from torch.utils.tensorboard import SummaryWriter
import torchvision

In [4]:
from Utils.dataset_utils import *
from Utils.visualization_utils import *

In [5]:
from torchsummary import summary
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

In [6]:
DATASET_2D_BASE_PATH=Path(r'C:\Users\lm3088\Documents\POM-CTproject\Muhammad\MiceCT_2Daxials\Axials')
WIDTH=320
HEIGHT=320

In [7]:


def get_training_augmentation():
    train_transform = [

        albu.Resize(HEIGHT, WIDTH),
        albu.ShiftScaleRotate(scale_limit=0.20, rotate_limit=30, shift_limit=0.1, p=1, border_mode=cv2.BORDER_CONSTANT),
        albu.RandomCrop(height=320, width=320),
        albu.Blur(blur_limit=3, p=0.4),
        albu.GaussNoise(p=0.5),
        albu.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
        albu.RandomBrightness(p=0.75)

    ]
    return albu.Compose(train_transform)


def get_validation_augmentation():
    test_transform = [
        albu.Resize(512, 512)
    ]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform

    Args:
        preprocessing_fn (callbale): data normalization function
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose

    """

    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

In [8]:
df_train=load_data(DATASET_2D_BASE_PATH/'train')
df_val=load_data(DATASET_2D_BASE_PATH/'val')
df_train.head()

,images,masks
0,C:\Users\lm3088\Documents\POM-CTproject\Muhamm...,C:\Users\lm3088\Documents\POM-CTproject\Muhamm...
1,C:\Users\lm3088\Documents\POM-CTproject\Muhamm...,C:\Users\lm3088\Documents\POM-CTproject\Muhamm...
2,C:\Users\lm3088\Documents\POM-CTproject\Muhamm...,C:\Users\lm3088\Documents\POM-CTproject\Muhamm...
3,C:\Users\lm3088\Documents\POM-CTproject\Muhamm...,C:\Users\lm3088\Documents\POM-CTproject\Muhamm...
4,C:\Users\lm3088\Documents\POM-CTproject\Muhamm...,C:\Users\lm3088\Documents\POM-CTproject\Muhamm...


### Training

In [9]:
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation
DEVICE = 'cuda'
TRAIN_RUNS_PATH=r'C:\Users\lm3088\Documents\GitHub\MicroCTsegmentation\runs'
MODEL_NAME='Unet'
BATCH_SIZE=8
EPOCHS=100


In [10]:
HUMAN_WEIGHTS_PATH=r'C:\Users\lm3088\Documents\POM-CTproject\Muhammad\HumanCT_2Daxials\TrainedWeights-HumanHeart'

In [11]:
ENCODERS = ['se_resnext50_32x4d','resnet101','efficientnet-b4']
# ENCODERS = ['se_resnext50_32x4d']

In [14]:
for j in range(len(ENCODERS)):
    
#     model = smp.Unet(
#     encoder_name=ENCODERS[j], 
#     encoder_weights=ENCODER_WEIGHTS, 
#     classes=1, 
#     activation=ACTIVATION,
#     )
  

    model=torch.load(os.path.join(HUMAN_WEIGHTS_PATH,f'Unet_{ENCODERS[j]}','best.pt'))
    preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODERS[j], ENCODER_WEIGHTS)
    
    WEIGHTS_PATH=os.path.join(TRAIN_RUNS_PATH,f'{MODEL_NAME}_{ENCODERS[j]}')
    if os.path.exists(WEIGHTS_PATH)==False:
        os.mkdir(WEIGHTS_PATH)
    else:
        print(f"Warning! Directory {WEIGHTS_PATH } already exists")
    
    train_dataset = Dataset(
    df_train['images'], 
    df_train['masks'], 
    augmentation=get_training_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn)
    )

    valid_dataset = Dataset(
        df_val['images'], 
        df_val['masks'], 
        augmentation=get_validation_augmentation(), 
        preprocessing=get_preprocessing(preprocessing_fn)
    ) 

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    loss = smp.utils.losses.DiceLoss()
    metrics = [
        smp.utils.metrics.IoU(threshold=0.5),
        smp.utils.metrics.Fscore()

    ]

    optimizer = torch.optim.Adam([ 
        dict(params=model.parameters(), lr=0.0001),
    ])
    train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
    )

    valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
    )
    min_loss = 100000000
    train_history=defaultdict(list)
    valid_history=defaultdict(list)
    
    comment = f'model = {ENCODERS[j]}'
    tb = SummaryWriter(comment)
#     images, labels = next(iter(train_loader))
#     grid = torchvision.utils.make_grid(images)
#     images=images.to(DEVICE)
#     tb.add_image("images", grid)
#     tb.add_graph(model, images)
#     tb.close()
    for i in range(0, EPOCHS):

        print('\nEpoch: {}'.format(i))
        train_logs = train_epoch.run(train_loader)
        valid_logs = valid_epoch.run(valid_loader)

        # do something (save model, change lr, etc.)
        if min_loss > valid_logs['dice_loss']:
            min_loss = valid_logs['dice_loss']
            torch.save(model, os.path.join(WEIGHTS_PATH,f'best_{str(i)}_{round(min_loss,4)}.pt'))
            print('Model saved!')

        if i == 25:
            optimizer.param_groups[0]['lr'] = 1e-5
            print('Decrease decoder learning rate to 1e-5!')
        # Maintain History
        for log_key in train_logs.keys():
            train_history[log_key].append(train_logs[log_key])
            valid_history[log_key].append(valid_logs[log_key])
            tb.add_scalar(f'train_{log_key}',train_logs[log_key], i)
            tb.add_scalar(f'validation_{log_key}',valid_logs[log_key], i)
        
#     tb.add_hparams(
#         {"model": ENCODERS[j]},{"min_loss": min_loss}
#     )
     
    del model
    
    pd.DataFrame(valid_history).to_csv(os.path.join(WEIGHTS_PATH,'validation_logs.csv'))
    pd.DataFrame(train_history).to_csv(os.path.join(WEIGHTS_PATH,'train_logs.csv'))
    print(f"{WEIGHTS_PATH} Completed!!!")
tb.close()

Warning! Directory C:\Users\lm3088\Documents\GitHub\MicroCTsegmentation\runs\Unet_se_resnext50_32x4d already exists

Epoch: 15
valid: 100%|████████████████| 192/192 [00:46<00:00,  4.17it/s, dice_loss - 0.3704, iou_score - 0.5602, fscore - 0.6219]
Model saved!

Epoch: 16
valid: 100%|████████████████| 192/192 [00:44<00:00,  4.35it/s, dice_loss - 0.4478, iou_score - 0.4895, fscore - 0.5478]

Epoch: 17
valid: 100%|████████████████| 192/192 [00:43<00:00,  4.39it/s, dice_loss - 0.2327, iou_score - 0.6998, fscore - 0.7579]
Model saved!

Epoch: 18
valid: 100%|████████████████| 192/192 [00:42<00:00,  4.56it/s, dice_loss - 0.2796, iou_score - 0.6543, fscore - 0.7116]

Epoch: 19
valid: 100%|████████████████| 192/192 [00:42<00:00,  4.56it/s, dice_loss - 0.4307, iou_score - 0.5056, fscore - 0.5636]

Epoch: 20
valid: 100%|█████████████████| 192/192 [00:42<00:00,  4.49it/s, dice_loss - 0.2738, iou_score - 0.6605, fscore - 0.716]

Epoch: 21
valid: 100%|█████████████████| 192/192 [00:43<00:00,  4.43it/

valid: 100%|█████████████████| 192/192 [00:41<00:00,  4.58it/s, dice_loss - 0.1803, iou_score - 0.755, fscore - 0.8086]

Epoch: 47
valid: 100%|████████████████| 192/192 [00:41<00:00,  4.60it/s, dice_loss - 0.2319, iou_score - 0.7021, fscore - 0.7554]

Epoch: 48
valid: 100%|█████████████████| 192/192 [00:41<00:00,  4.59it/s, dice_loss - 0.209, iou_score - 0.7223, fscore - 0.7755]

Epoch: 49
valid: 100%|████████████████| 192/192 [00:41<00:00,  4.59it/s, dice_loss - 0.1789, iou_score - 0.7544, fscore - 0.8081]

Epoch: 50
valid: 100%|█████████████████| 192/192 [00:41<00:00,  4.59it/s, dice_loss - 0.1874, iou_score - 0.7523, fscore - 0.806]

Epoch: 51
valid: 100%|████████████████| 192/192 [00:41<00:00,  4.59it/s, dice_loss - 0.1815, iou_score - 0.7504, fscore - 0.8039]

Epoch: 52
valid: 100%|████████████████| 192/192 [00:42<00:00,  4.56it/s, dice_loss - 0.2035, iou_score - 0.7302, fscore - 0.7825]

Epoch: 53
valid: 100%|████████████████| 192/192 [00:42<00:00,  4.54it/s, dice_loss - 0.2195, 

valid: 100%|████████████████| 192/192 [00:41<00:00,  4.59it/s, dice_loss - 0.2411, iou_score - 0.6942, fscore - 0.7465]

Epoch: 79
valid: 100%|████████████████| 192/192 [00:41<00:00,  4.60it/s, dice_loss - 0.2607, iou_score - 0.6734, fscore - 0.7259]

Epoch: 80
valid: 100%|████████████████| 192/192 [00:41<00:00,  4.60it/s, dice_loss - 0.2309, iou_score - 0.7018, fscore - 0.7548]

Epoch: 81
valid: 100%|████████████████| 192/192 [00:41<00:00,  4.61it/s, dice_loss - 0.2189, iou_score - 0.7125, fscore - 0.7654]

Epoch: 82
valid: 100%|████████████████| 192/192 [00:41<00:00,  4.58it/s, dice_loss - 0.2547, iou_score - 0.6778, fscore - 0.7304]

Epoch: 83
valid: 100%|████████████████| 192/192 [00:41<00:00,  4.62it/s, dice_loss - 0.2603, iou_score - 0.6718, fscore - 0.7247]

Epoch: 84
valid: 100%|████████████████| 192/192 [00:41<00:00,  4.62it/s, dice_loss - 0.2599, iou_score - 0.6756, fscore - 0.7287]

Epoch: 85
valid: 100%|█████████████████| 192/192 [00:41<00:00,  4.62it/s, dice_loss - 0.243, 

valid: 100%|█████████████████| 192/192 [00:44<00:00,  4.29it/s, dice_loss - 0.216, iou_score - 0.7159, fscore - 0.7727]
Decrease decoder learning rate to 1e-5!

Epoch: 26
valid: 100%|████████████████| 192/192 [00:44<00:00,  4.28it/s, dice_loss - 0.1982, iou_score - 0.7346, fscore - 0.7889]

Epoch: 27
valid: 100%|████████████████| 192/192 [00:44<00:00,  4.28it/s, dice_loss - 0.1925, iou_score - 0.7407, fscore - 0.7947]

Epoch: 28
valid: 100%|█████████████████| 192/192 [00:44<00:00,  4.29it/s, dice_loss - 0.1879, iou_score - 0.7452, fscore - 0.799]

Epoch: 29
valid: 100%|█████████████████| 192/192 [00:44<00:00,  4.28it/s, dice_loss - 0.182, iou_score - 0.7497, fscore - 0.8036]
Model saved!

Epoch: 30
valid: 100%|████████████████| 192/192 [00:44<00:00,  4.28it/s, dice_loss - 0.1927, iou_score - 0.7402, fscore - 0.7944]

Epoch: 31
valid: 100%|█████████████████| 192/192 [00:44<00:00,  4.28it/s, dice_loss - 0.1924, iou_score - 0.739, fscore - 0.7932]

Epoch: 32
valid: 100%|████████████████| 

valid: 100%|████████████████| 192/192 [00:44<00:00,  4.29it/s, dice_loss - 0.1757, iou_score - 0.7598, fscore - 0.8142]

Epoch: 58
valid: 100%|████████████████| 192/192 [00:44<00:00,  4.28it/s, dice_loss - 0.1669, iou_score - 0.7695, fscore - 0.8237]
Model saved!

Epoch: 59
valid: 100%|████████████████| 192/192 [00:44<00:00,  4.28it/s, dice_loss - 0.1828, iou_score - 0.7495, fscore - 0.8036]

Epoch: 60
valid: 100%|████████████████| 192/192 [00:44<00:00,  4.29it/s, dice_loss - 0.1811, iou_score - 0.7513, fscore - 0.8053]

Epoch: 61
valid: 100%|█████████████████| 192/192 [00:44<00:00,  4.29it/s, dice_loss - 0.2007, iou_score - 0.731, fscore - 0.7848]

Epoch: 62
valid: 100%|████████████████| 192/192 [00:44<00:00,  4.29it/s, dice_loss - 0.1785, iou_score - 0.7534, fscore - 0.8077]

Epoch: 63
valid: 100%|██████████████████| 192/192 [00:44<00:00,  4.29it/s, dice_loss - 0.1824, iou_score - 0.75, fscore - 0.8038]

Epoch: 64
valid: 100%|████████████████| 192/192 [00:44<00:00,  4.29it/s, dice_lo

valid: 100%|████████████████| 192/192 [00:44<00:00,  4.29it/s, dice_loss - 0.1889, iou_score - 0.7443, fscore - 0.7976]

Epoch: 90
valid: 100%|████████████████| 192/192 [00:44<00:00,  4.29it/s, dice_loss - 0.1702, iou_score - 0.7615, fscore - 0.8153]

Epoch: 91
valid: 100%|████████████████| 192/192 [00:44<00:00,  4.29it/s, dice_loss - 0.1697, iou_score - 0.7621, fscore - 0.8157]

Epoch: 92
valid: 100%|█████████████████| 192/192 [00:44<00:00,  4.28it/s, dice_loss - 0.1724, iou_score - 0.7591, fscore - 0.813]

Epoch: 93
valid: 100%|████████████████| 192/192 [00:44<00:00,  4.28it/s, dice_loss - 0.1726, iou_score - 0.7587, fscore - 0.8129]

Epoch: 94
valid: 100%|████████████████| 192/192 [00:44<00:00,  4.29it/s, dice_loss - 0.1778, iou_score - 0.7562, fscore - 0.8101]

Epoch: 95
valid: 100%|████████████████| 192/192 [00:44<00:00,  4.28it/s, dice_loss - 0.1774, iou_score - 0.7571, fscore - 0.8107]

Epoch: 96
valid: 100%|████████████████| 192/192 [00:44<00:00,  4.28it/s, dice_loss - 0.1723, 

valid: 100%|██████████████████| 192/192 [00:43<00:00,  4.38it/s, dice_loss - 0.29, iou_score - 0.6447, fscore - 0.6993]

Epoch: 37
valid: 100%|████████████████| 192/192 [00:43<00:00,  4.39it/s, dice_loss - 0.2967, iou_score - 0.6392, fscore - 0.6935]

Epoch: 38
valid: 100%|████████████████| 192/192 [00:43<00:00,  4.38it/s, dice_loss - 0.2816, iou_score - 0.6611, fscore - 0.7151]

Epoch: 39
valid: 100%|████████████████| 192/192 [00:43<00:00,  4.38it/s, dice_loss - 0.2831, iou_score - 0.6511, fscore - 0.7051]

Epoch: 40
valid: 100%|█████████████████| 192/192 [00:43<00:00,  4.38it/s, dice_loss - 0.2806, iou_score - 0.6548, fscore - 0.709]

Epoch: 41
valid: 100%|████████████████| 192/192 [00:43<00:00,  4.38it/s, dice_loss - 0.2775, iou_score - 0.6542, fscore - 0.7084]

Epoch: 42
valid: 100%|████████████████| 192/192 [00:43<00:00,  4.38it/s, dice_loss - 0.2827, iou_score - 0.6529, fscore - 0.7068]

Epoch: 43
valid: 100%|████████████████| 192/192 [00:43<00:00,  4.39it/s, dice_loss - 0.2812, 

valid: 100%|████████████████| 192/192 [00:43<00:00,  4.38it/s, dice_loss - 0.2391, iou_score - 0.6972, fscore - 0.7501]

Epoch: 69
valid: 100%|████████████████| 192/192 [00:43<00:00,  4.39it/s, dice_loss - 0.2418, iou_score - 0.6897, fscore - 0.7429]

Epoch: 70
valid: 100%|████████████████| 192/192 [00:43<00:00,  4.37it/s, dice_loss - 0.2396, iou_score - 0.6959, fscore - 0.7489]

Epoch: 71
valid: 100%|████████████████| 192/192 [00:43<00:00,  4.40it/s, dice_loss - 0.1985, iou_score - 0.7379, fscore - 0.7908]

Epoch: 72
valid: 100%|████████████████| 192/192 [00:43<00:00,  4.39it/s, dice_loss - 0.2635, iou_score - 0.6712, fscore - 0.7239]

Epoch: 73
valid: 100%|█████████████████| 192/192 [00:43<00:00,  4.39it/s, dice_loss - 0.2472, iou_score - 0.6864, fscore - 0.739]

Epoch: 74
valid: 100%|█████████████████| 192/192 [00:43<00:00,  4.40it/s, dice_loss - 0.2443, iou_score - 0.6924, fscore - 0.745]

Epoch: 75
valid: 100%|████████████████| 192/192 [00:43<00:00,  4.39it/s, dice_loss - 0.2078, 